In [ ]:
import rawpy
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO
import sys
from dotenv import load_dotenv
import os
from pathlib import Path
load_dotenv()

# --- CONFIGURATION ---
# Le modèle 'yolov8s-seg.pt' (small) est un bon compromis vitesse/précision.
# Il sera téléchargé automatiquement au premier lancement.
MODELE_IA = 'yolov8s-seg.pt'
# ID de la classe "oiseau" dans le jeu de données COCO utilisé par YOLO
CLASS_ID_OISEAU = 14 

def charger_et_developper_raf_16bit(chemin_raf):
    """
    Ouvre le fichier .RAF et le convertit en une image RGB 16 bits (0-65535).
    Le 16 bits retient beaucoup plus d'informations de couleur que le 8 bits standard.
    """
    print(f"-> 1. Lecture et développement du fichier RAW (16-bit) : {chemin_raf}")
    try:
        with rawpy.imread(chemin_raf) as raw:
            # output_bps=16 est crucial pour la haute qualité
            # use_camera_wb=True utilise la balance des blancs de l'appareil
            rgb_16bit = raw.postprocess(
                use_camera_wb=True, 
                bright=1.0,          # Luminosité de base
                user_sat=None,       # Saturation par défaut
                no_auto_bright=False,# Laisser rawpy ajuster légèrement l'exposition
                output_bps=16        # Sortie en 16 bits par canal
            )
        return rgb_16bit
    except Exception as e:
        print(f"Erreur lors du chargement du RAW : {e}")
        sys.exit(1)

def generer_masque_oiseau_ia(image_rgb_16bit):
    """
    Utilise YOLOv8 pour trouver les oiseaux et générer un masque de segmentation.
    """
    print(f"-> 2. Analyse IA (YOLOv8) pour trouver l'oiseau...")
    
    # L'IA travaille généralement en 8-bit. On crée une copie 8-bit juste pour l'analyse.
    # On normalise de 0-65535 vers 0-255.
    img_8bit_pour_ia = (image_rgb_16bit / 256).astype('uint8')
    
    # Chargement du modèle
    model = YOLO(MODELE_IA)
    
    # Exécution de l'inférence (détection et segmentation)
    # verbose=False réduit le texte dans la console
    results = model.predict(img_8bit_pour_ia, verbose=False, conf=0.4)
    result = results[0]

    h, w = image_rgb_16bit.shape[:2]
    masque_final = np.zeros((h, w), dtype=np.uint8)
    oiseau_trouve = False

    if result.masks is not None:
        # On parcourt les objets détectés
        for i, box in enumerate(result.boxes):
            class_id = int(box.cls[0])
            # Si c'est un oiseau (ID 14)
            if class_id == CLASS_ID_OISEAU:
                oiseau_trouve = True
                # Récupérer le masque brut (souvent de petite taille)
                mask_data = result.masks.data[i].cpu().numpy()
                # Redimensionner le masque à la taille de l'image originale
                mask_resized = cv2.resize(mask_data, (w, h), interpolation=cv2.INTER_LINEAR)
                # Ajouter ce masque au masque principal (au cas où il y a plusieurs oiseaux)
                masque_final = np.maximum(masque_final, mask_resized)

    if not oiseau_trouve:
        print("ATTENTION : Aucun oiseau détecté par l'IA. L'amélioration ciblée sera ignorée.")
        # On retourne un masque noir, le script continuera avec les améliorations globales.
        return masque_final

    print("   Oiseau(x) identifié(s) et masqué(s) avec succès.")
    # Le masque est actuellement en float entre 0.0 et 1.0, on le passe en 0-255 uint8
    return (masque_final * 255).astype(np.uint8)

def traiter_image_haute_qualite(image_rgb_16bit, mask_8bit):
    """
    Version corrigée pour gérer la limitation 16-bit d'OpenCV dans Lab.
    """
    print("-> 3. Application des traitements (Correction 16-bit -> Float32)...")
    
    # 1. Conversion RGB vers BGR
    img_bgr_16 = cv2.cvtColor(image_rgb_16bit, cv2.COLOR_RGB2BGR)

    # --- A. NETTETÉ CIBLÉE (Unsharp Masking) ---
    blurred_16 = cv2.GaussianBlur(img_bgr_16, (0, 0), sigmaX=3)
    sharpened_16 = cv2.addWeighted(img_bgr_16, 1.8, blurred_16, -0.8, 0)

    # Mixage avec le masque
    mask_3c_norm = cv2.cvtColor(mask_8bit, cv2.COLOR_GRAY2BGR).astype(np.float32) / 255.0
    img_bgr_16_float = img_bgr_16.astype(np.float32)
    sharpened_16_float = sharpened_16.astype(np.float32)
    
    merged_float = (sharpened_16_float * mask_3c_norm) + (img_bgr_16_float * (1.0 - mask_3c_norm))
    # On garde merged_float pour la suite au lieu de repasser en uint16 tout de suite

    # --- B. AMÉLIORATION GLOBALE (Luminosité/Contraste via Lab) ---
    # Normalisation pour cvtColor : 16-bit uint16 (0-65535) -> float32 (0.0-1.0)
    img_normalized = merged_float / 65535.0
    
    # Maintenant, cvtColor accepte le float32 pour le passage en Lab
    lab_f32 = cv2.cvtColor(img_normalized, cv2.COLOR_BGR2Lab)
    l_channel, a_channel, b_channel = cv2.split(lab_f32)

    # Le canal L en float32 Lab va de 0.0 à 100.0. 
    # Pour utiliser CLAHE (qui aime les entiers), on le convertit temporairement en uint16 (0-65535)
    l_u16 = (l_channel * (65535.0 / 100.0)).astype(np.uint16)

    # Application du CLAHE sur le canal L
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    l_enhanced_u16 = clahe.apply(l_u16)

    # On repasse le canal L en float32 (0.0-100.0)
    l_enhanced_f32 = l_enhanced_u16.astype(np.float32) * (100.0 / 65535.0)

    # Fusion et retour en BGR (toujours en float32 0.0-1.0)
    lab_final_f32 = cv2.merge((l_enhanced_f32, a_channel, b_channel))
    final_bgr_f32 = cv2.cvtColor(lab_final_f32, cv2.COLOR_Lab2BGR)

    # Re-conversion finale en 16-bit uint16 (0-65535) pour la sauvegarde TIFF
    final_bgr_16 = np.clip(final_bgr_f32 * 65535.0, 0, 65535).astype(np.uint16)

    return final_bgr_16

def sauvegarder_tiff_hq(image_bgr_16bit, chemin_sortie):
    """
    Sauvegarde en TIFF 16-bit haute qualité en utilisant OpenCV.
    OpenCV gère mieux les tableaux NumPy 16-bit multi-canaux pour le format TIFF.
    """
    print(f"-> 4. Sauvegarde du TIFF 16-bit (Haute Qualité) : {chemin_sortie}")
    
    try:
        # Contrairement à Pillow, OpenCV attend du BGR. 
        # Comme notre image 'resultat_bgr_16' est déjà en BGR, 
        # on peut l'enregistrer directement.
        
        # Configuration des options TIFF (facultatif mais recommandé pour la qualité)
        # 1 = pas de compression, 5 = LZW (sans perte), 32946 = Deflate (sans perte)
        params = [cv2.IMWRITE_TIFF_COMPRESSION, 32946] 
        
        succes = cv2.imwrite(chemin_sortie, image_bgr_16bit, params)
        
        if succes:
            print(f"Fichier enregistré avec succès en 16-bit !")
        else:
            print("Échec de l'enregistrement du fichier.")
            
    except Exception as e:
        print(f"Erreur lors de la sauvegarde : {e}")


In [17]:
if __name__ == "__main__":
    base_path = os.getenv("BASE_PATH")
    fichier_entree_raf = f"{base_path}data/inputs/DSCF3552.RAF"
    fichier_sortie_tiff = f"{base_path}data/outputs/DSCF3552_postttt.tiff" 

    # 1. Charger le RAW
    rgb_16 = charger_et_developper_raf_16bit(fichier_entree_raf)
    
    # 2. IA : Masque de l'oiseau
    masque_ia = generer_masque_oiseau_ia(rgb_16)
    
    # 3. Traitement (Netteté ciblée + Lab CLAHE corrigé)
    # Assurez-vous d'utiliser la version "float32" de cette fonction donnée précédemment
    resultat_bgr_16 = traiter_image_haute_qualite(rgb_16, masque_ia)
    
    # 4. Sauvegarde directe via OpenCV
    sauvegarder_tiff_hq(resultat_bgr_16, fichier_sortie_tiff)

-> 1. Lecture et développement du fichier RAW (16-bit) : C:/Users/savma/Projects/ttt_image/data/inputs/DSCF3552.RAF
-> 2. Analyse IA (YOLOv8) pour trouver l'oiseau...
   Oiseau(x) identifié(s) et masqué(s) avec succès.
-> 3. Application des traitements (Correction 16-bit -> Float32)...
-> 4. Sauvegarde du TIFF 16-bit (Haute Qualité) : C:/Users/savma/Projects/ttt_image/data/outputs/DSCF3552_postttt.tiff
✅ Fichier enregistré avec succès en 16-bit !
